In [1]:
!pip install -q -U keras-tuner
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, LSTM, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import save_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from google.colab import output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 13.9 MB/s eta 0:00:00


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul  1 16:22:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
set_name='flap_jump'
split = 'PS'
labels = 2

set_folder = 'cv_'+set_name+'_'+split+'_tuner'
cv_set_folder='cv_sets'
train_set_name = set_name+'_train_set_'+split+'_cv_'
test_set_name = set_name+'_test_set_'+split+'_cv_'
os.chdir("/content/drive/My Drive/promotion/data/"+set_folder)
print(os.getcwd())
sequence_length = 15
np.random.seed(42)
column_names_labels = ['Flattern', 'Hüpfen']
column_names_features = ['Shoulder_x_0', 'Shoulder_y_0', 'Elbow_x_0', 'Elbow_y_0',
                     'RWrist_x_0', 'RWrist_y_0', 'LShoulder_x_0', 'LShoulder_y_0',
                     'LElbow_x_0', 'LElbow_y_0', 'LWrist_x_0', 'LWrist_y_0',
                     'Shoulder_x_1', 'Shoulder_y_1', 'Elbow_x_1', 'Elbow_y_1',
                     'RWrist_x_1', 'RWrist_y_1', 'LShoulder_x_1', 'LShoulder_y_1',
                     'LElbow_x_1', 'LElbow_y_1', 'LWrist_x_1', 'LWrist_y_1',
                     'Shoulder_x_2', 'Shoulder_y_2', 'Elbow_x_2', 'Elbow_y_2',
                     'RWrist_x_2', 'RWrist_y_2', 'LShoulder_x_2', 'LShoulder_y_2',
                     'LElbow_x_2', 'LElbow_y_2', 'LWrist_x_2', 'LWrist_y_2',
                     'Shoulder_x_3', 'Shoulder_y_3', 'Elbow_x_3', 'Elbow_y_3',
                     'RWrist_x_3','RWrist_y_3', 'LShoulder_x_3', 'LShoulder_y_3',
                     'LElbow_x_3', 'LElbow_y_3', 'LWrist_x_3', 'LWrist_y_3',
                     'Shoulder_x_4', 'Shoulder_y_4', 'Elbow_x_4', 'Elbow_y_4',
                     'RWrist_x_4', 'RWrist_y_4', 'LShoulder_x_4', 'LShoulder_y_4',
                     'LElbow_x_4', 'LElbow_y_4', 'LWrist_x_4', 'LWrist_y_4']

/content/drive/My Drive/promotion/data/cv_flap_jump_PS_tuner


In [7]:
train_df_pre = pd.read_csv("/content/drive/My Drive/promotion/data/"+cv_set_folder+'/'+train_set_name+str(1), sep=';')
test_df_pre = pd.read_csv("/content/drive/My Drive/promotion/data/"+cv_set_folder+'/'+test_set_name+str(1), sep=';')

def reshape_sets(train_df_pre, test_df_pre, column_names_labels, column_names_drop):
    train_df_labels = train_df_pre.loc[:, column_names_labels]
    test_df_labels = test_df_pre.loc[:, column_names_labels]
    train_df = train_df_pre.loc[:, column_names_features]
    test_df = test_df_pre.loc[:, column_names_features]
    x_train = np.asarray(train_df)
    x_test = np.asarray(test_df)
    y_train = np.asarray(train_df_labels)
    y_test = np.asarray(test_df_labels)
    train_samples = int(x_train.shape[0]/sequence_length)
    features = x_train.shape[1]
    test_samples = int(x_test.shape[0]/sequence_length)
    x_train = x_train.reshape((train_samples,sequence_length,features))
    x_test = x_test.reshape((test_samples,sequence_length,features))
    y_train = y_train.reshape((train_samples,sequence_length,labels))
    y_test = y_test.reshape((test_samples,sequence_length,labels))
    y_train = y_train[:,0,:]
    y_test = y_test[:,0,:]
    print("x_train_shape: "+str(x_train.shape))
    print("x_test_shape: "+str(x_test.shape))
    print("y_train_shape: "+str(y_train.shape))
    print("y_test_shape: "+str(y_test.shape))
    print(column_names_labels[0]+" y_train: "+str(round(y_train[:,0].sum()/y_train[:,0].size*100,2))+" ("+
          str(round(y_train[:,0].sum()/(y_train[:,0].sum()+y_test[:,0].sum()),2))+")")
    print(column_names_labels[1]+" y_train: "+str(round(y_train[:,1].sum()/y_train[:,1].size*100,2))+" ("+
          str(round(y_train[:,1].sum()/(y_train[:,1].sum()+y_test[:,1].sum()),2))+")")
    print(column_names_labels[0]+" y_test: "+str(round(y_test[:,0].sum()/y_test[:,0].size*100,2))+" ("+
          str(round(y_test[:,0].sum()/(y_train[:,0].sum()+y_test[:,0].sum()),2))+")")
    print(column_names_labels[1]+" y_test: "+str(round(y_test[:,1].sum()/y_test[:,1].size*100,2))+" ("+
          str(round(y_test[:,1].sum()/(y_train[:,1].sum()+y_test[:,1].sum()),2))+")")
    return x_train, x_test, y_train, y_test, features

x_train, x_test, y_train, y_test, features = reshape_sets(train_df_pre, test_df_pre
                                    , column_names_labels, column_names_features)


x_train_shape: (1040, 15, 60)
x_test_shape: (820, 15, 60)
y_train_shape: (1040, 2)
y_test_shape: (820, 2)
Flattern y_train: 44.33 (0.56)
Hüpfen y_train: 21.83 (0.58)
Flattern y_test: 43.66 (0.44)
Hüpfen y_test: 19.88 (0.42)


In [8]:
print('test_set: ' + str(x_test.shape[0]/(x_test.shape[0]+x_train.shape[0])*100))

test_set: 44.086021505376344


In [9]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(LSTM(hp.Int('input_unit',min_value=64,max_value=448,step=32),return_sequences=True, input_shape=(sequence_length,features)))
        for i in range(hp.Int('n_layers', 1, 3)):
            model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=64,max_value=448,step=32),return_sequences=True))
        model.add(LSTM(hp.Int('layer_2_neurons',min_value=64,max_value=448,step=32)))
        model.add(Dropout(hp.Float('Dropout_rate',min_value=0.0,max_value=0.3,step=0.1)))
        model.add(Dense(y_train.shape[1], kernel_initializer='uniform', activation='sigmoid'))
        hp_learning_rate = hp.Choice('learning_rate', values=[0.0003, 0.001, 0.01])
        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                      metrics=[keras.metrics.BinaryAccuracy()])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [16, 32, 64]),
            **kwargs,
        )

In [10]:
tuner = kt.Hyperband(
    MyHyperModel(),
    objective='binary_accuracy',
    max_epochs=1000,
    #overwrite=True,
    hyperband_iterations=1,
    project_name='kt')

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
input_unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 448, 'step': 32, 'sampling': 'linear'}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
lstm_0_units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 448, 'step': 32, 'sampling': 'linear'}
layer_2_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 448, 'step': 32, 'sampling': 'linear'}
Dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.0003, 'conditions': [], 'values': [0.0003, 0.001, 0.01], 'ordered': True}
lstm_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 448, 'step': 32, 'sampling': 'linear'}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
lstm

In [12]:
es = tf.keras.callbacks.EarlyStopping(patience=100)
tb = keras.callbacks.TensorBoard("/content/drive/My Drive/promotion/data/"+set_folder+"/tmp/tb_logs")

In [13]:
tuner.search(x=x_train,
             y=y_train,
             validation_data=(x_test,y_test),
             epochs=1000,
             callbacks=[es, tb])

Trial 2074 Complete [00h 01m 01s]
binary_accuracy: 0.8182692527770996

Best binary_accuracy So Far: 0.9418269395828247
Total elapsed time: 02h 08m 19s


In [14]:
tuner.results_summary()

Results summary
Results in ./kt
Showing 10 best trials
Objective(name="binary_accuracy", direction="max")

Trial 1934 summary
Hyperparameters:
input_unit: 224
n_layers: 1
lstm_0_units: 192
layer_2_neurons: 224
Dropout_rate: 0.1
learning_rate: 0.0003
lstm_1_units: 288
batch_size: 16
lstm_2_units: 224
tuner/epochs: 1000
tuner/initial_epoch: 334
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 1930
Score: 0.9418269395828247

Trial 1738 summary
Hyperparameters:
input_unit: 288
n_layers: 1
lstm_0_units: 352
layer_2_neurons: 416
Dropout_rate: 0.0
learning_rate: 0.0003
lstm_1_units: 64
batch_size: 32
lstm_2_units: 96
tuner/epochs: 1000
tuner/initial_epoch: 334
tuner/bracket: 5
tuner/round: 5
tuner/trial_id: 1735
Score: 0.9408653974533081

Trial 2047 summary
Hyperparameters:
input_unit: 256
n_layers: 1
lstm_0_units: 256
layer_2_neurons: 416
Dropout_rate: 0.1
learning_rate: 0.0003
lstm_1_units: 160
batch_size: 32
lstm_2_units: 352
tuner/epochs: 1000
tuner/initial_epoch: 334
tuner/bracket: 2
tune

In [15]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [16]:
print('input_unit: '+ str(best_hps.get('input_unit')))
print('n_layers: '+ str(best_hps.get('n_layers')))
print('lstm_0_units: '+ str(best_hps.get('lstm_0_units')))
print('layer_2_neurons: '+ str(best_hps.get('layer_2_neurons')))
print('Dropout_rate: '+ str(best_hps.get('Dropout_rate')))
print('learning_rate: '+ str(best_hps.get('learning_rate')))
print('lstm_1_units: '+ str(best_hps.get('lstm_1_units')))
print('lstm_2_units: '+ str(best_hps.get('lstm_2_units')))
print('batch_size: '+ str(best_hps.get('batch_size')))

input_unit: 224
n_layers: 1
lstm_0_units: 192
layer_2_neurons: 224
Dropout_rate: 0.1
learning_rate: 0.0003
lstm_1_units: 288
lstm_2_units: 224
batch_size: 16


In [17]:
# get best model
best_model = tuner.get_best_models(num_models=1)[0]

In [18]:
# save
save_model(best_model,'best_model.h5')

In [19]:
from sklearn.metrics import multilabel_confusion_matrix

#load best model (early stopping)
best_model = load_model('best_model.h5')
# summarize model.
best_model.summary()
# evaluate the model
scores_best_model = best_model.evaluate(x_test, y_test, verbose=0)
results_best_model = {}
#yhat_classes = best_model.predict_classes(x_test, verbose=0)
#yhat_classes = (best_model.predict(x_test) > 0.5).astype("int32")
# make a prediction on the test set
yhat = best_model.predict(x_test, verbose=0)
# round probabilities to class labels
yhat_round = yhat.round()
# multilabel flap + jump
yhat_flap = yhat_round[:,:1] # the first : to get all rows, and :1 to get first column
yhat_jump = yhat_round[:,1:2]
y_test_flap = y_test[:,:1]
y_test_jump = y_test[:,1:2]
# flap
# accuracy: (tp + tn) / (p + n)
accuracy_flap = accuracy_score(y_test_flap, yhat_flap)
print('Accuracy flap: %f' % accuracy_flap)
results_best_model['accuracy_flap'] = accuracy_flap
# precision tp / (tp + fp)
precision_flap = precision_score(y_test_flap, yhat_flap)
print('Precision flap: %f' % precision_flap)
results_best_model['precision_flap'] = precision_flap
# recall: tp / (tp + fn)
recall_flap = recall_score(y_test_flap, yhat_flap)
print('Recall flap: %f' % recall_flap)
results_best_model['recall_flap'] = recall_flap
# f1: 2 tp / (2 tp + fp + fn)
f1_flap = f1_score(y_test_flap, yhat_flap)
print('F1 score flap: %f' % f1_flap)
results_best_model['f1_flap'] = f1_flap
#multilabel_confusion_matrix(y_true, y_pred)
confusion_matrix_flap = multilabel_confusion_matrix(y_test_flap, yhat_flap)
# jump
# accuracy: (tp + tn) / (p + n)
accuracy_jump = accuracy_score(y_test_jump, yhat_jump)
print('Accuracy jump: %f' % accuracy_jump)
results_best_model['accuracy_jump'] = accuracy_jump
# precision tp / (tp + fp)
precision_jump = precision_score(y_test_jump, yhat_jump)
print('Precision jump: %f' % precision_jump)
results_best_model['precision_jump'] = precision_jump
# recall: tp / (tp + fn)
recall_jump = recall_score(y_test_jump, yhat_jump)
print('Recall jump: %f' % recall_jump)
results_best_model['recall_jump'] = recall_jump
# f1: 2 tp / (2 tp + fp + fn)
f1_jump = f1_score(y_test_jump, yhat_jump)
print('F1 score jump: %f' % f1_jump)
results_best_model['f1_jump'] = f1_jump
#multilabel_confusion_matrix(y_true, y_pred)
confusion_matrix_jump = multilabel_confusion_matrix(y_test_jump, yhat_jump)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 224)           255360    
                                                                 
 lstm_1 (LSTM)               (None, 15, 192)           320256    
                                                                 
 lstm_2 (LSTM)               (None, 224)               373632    
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 2)                 450       
                                                                 
Total params: 949,698
Trainable params: 949,698
Non-trainable params: 0
_________________________________________________________________


Accuracy flap: 0.619512
Precision flap: 0.579861
Recall flap: 0.466480
F1 score flap: 0.517028
Accuracy jump: 0.693902
Precision jump: 0.238095
Recall jump: 0.245399
F1 score jump: 0.241692


In [20]:
# save data
# results
results_best_model_df = pd.DataFrame(list(results_best_model.items()), columns = ['measure','value'])
results_best_model_df.to_csv('results_best_model', sep=';', index=False)
# confusion matrix flap
confusion = pd.DataFrame(confusion_matrix_flap[:,0,:])
confusion.to_csv('confusion_matrix_best_model_flap', sep=';', index=False)
# confusion matrix jump
confusion = pd.DataFrame(confusion_matrix_jump[:,0,:])
confusion.to_csv('confusion_matrix_best_model_jump', sep=';', index=False)

In [21]:
tuner.oracle.get_best_trials()[0].trial_id

'1934'

In [22]:
#multilabel_confusion_matrix(y_true, y_pred)
#cm = skm.multilabel_confusion_matrix(y_true, y_pred)
#print(skm.classification_report(y_true,y_pred))
cm = multilabel_confusion_matrix(y_test, yhat_round)
print(cm)
from sklearn.metrics import classification_report
print(classification_report(y_test,yhat_round,target_names=['flap', 'jump']))

[[[341 121]
  [191 167]]

 [[529 128]
  [123  40]]]
              precision    recall  f1-score   support

        flap       0.58      0.47      0.52       358
        jump       0.24      0.25      0.24       163

   micro avg       0.45      0.40      0.42       521
   macro avg       0.41      0.36      0.38       521
weighted avg       0.47      0.40      0.43       521
 samples avg       0.17      0.19      0.17       521



In [23]:
cm_flap = cm[0]
cm_jump = cm[1]
print(cm_flap)
print(cm_jump)

[[341 121]
 [191 167]]
[[529 128]
 [123  40]]


In [24]:
# save confusion matrix
# confusion matrix flap
confusion = pd.DataFrame(cm_flap)
confusion.to_csv('confusion_matrix_best_model_flap', sep=';', index=False)
# confusion matrix jump
confusion = pd.DataFrame(cm_jump)
confusion.to_csv('confusion_matrix_best_model_jump', sep=';', index=False)

In [25]:
#save classification report
report = classification_report(y_test,yhat_round,target_names=['flap', 'jump'],output_dict=True)
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_best_model', sep=';', index=False)